# Tool Calling in OpenDXA

This tutorial covers how to make resource methods callable by LLMs through the tool calling system in OpenDXA. Tool calling is specifically designed for classes that inherit from `BaseResource` - it's a core feature that enables LLMs in both the planning and reasoning layers to interact with your resources in a structured way.

## Learning Objectives

By the end of this tutorial, you will understand:

1. What tool calling is and how it works with resources
2. How to use the `tool_callable` decorator in your resource classes
3. How to structure tool parameters and schemas
4. Best practices for implementing callable resource methods
5. How to test and debug tool calls

## Prerequisites

- Basic understanding of OpenDXA's 2-layer architecture (planning and reasoning)
- Familiarity with Python decorators and type hints
- Understanding of basic resource management concepts

## 1. Understanding Tool Calling

Tool calling allows LLMs in both the planning and reasoning layers to interact with resources in a structured way. This is achieved through:

1. **Decorators**: The `@tool_callable` decorator marks methods as callable by LLMs
2. **Schemas**: Parameter schemas define the structure of tool inputs
3. **Validation**: Input validation ensures correct tool usage
4. **Error Handling**: Proper error handling for tool execution

Let's start with a basic example:

In [ ]:
from typing import Any

from opendxa import BaseResource, tool_callable


class DataAnalysisResource(BaseResource):
    """Example resource with tool-callable methods."""

    @tool_callable(
        name="analyze_data",
        description="Analyze data using statistical methods",
        parameters={
            "data": {"type": "array", "description": "List of numerical values"},
            "method": {"type": "string", "description": "Analysis method to use"},
        },
    )
    async def analyze_data(self, data: list[float], method: str = "mean") -> dict[str, Any]:
        """Analyze data using specified method."""
        if method == "mean":
            result = sum(data) / len(data)
        elif method == "sum":
            result = sum(data)
        else:
            raise ValueError(f"Unknown method: {method}")

        return {"result": result, "method": method}

    @tool_callable(
        name="visualize_data",
        description="Create visualization of data",
        parameters={
            "data": {"type": "array", "description": "Data to visualize"},
            "chart_type": {"type": "string", "description": "Type of chart to create"},
        },
    )
    async def visualize_data(self, data: list[float], chart_type: str = "line") -> dict[str, Any]:
        """Create visualization of data."""
        return {"chart_type": chart_type, "data_points": len(data), "status": "success"}

## 2. Using Tool Calling in Planning and Reasoning

Let's see how tool calling is used in both the planning and reasoning layers:

In [ ]:
from opendxa import ChainOfThoughtStrategy, ExecutionContext, Plan, PlanStep

# Create resource instance
analysis_resource = DataAnalysisResource()

# Create execution context
context = ExecutionContext()

# Register resource
context.register_resource(analysis_resource)

# Create a plan that uses the tool
plan = Plan(
    name="data_analysis_plan", description="Plan for analyzing and visualizing data", objective="Analyze and visualize manufacturing data"
)

# Add plan steps that use the tools
plan.add_step(
    PlanStep(
        name="analyze_manufacturing_data", description="Analyze manufacturing data", tool="analyze_data", tool_params={"method": "mean"}
    )
)

plan.add_step(
    PlanStep(
        name="visualize_results",
        description="Create visualization of analysis results",
        tool="visualize_data",
        tool_params={"chart_type": "line"},
    )
)

# Create reasoning strategy
reasoning_strategy = ChainOfThoughtStrategy()

# Execute the plan
result = context.execute_plan(plan, reasoning_strategy)

print("Plan Execution Result:")
print(f"- Status: {result.status}")
print(f"- Completed Steps: {result.completed_steps}")
print(f"- Total Duration: {result.total_duration}")

## 3. Advanced Tool Calling Features

Let's explore some advanced features of tool calling:

In [ ]:
from pydantic import BaseModel

from opendxa import ToolSchema


# Define parameter models
class AnalysisParameters(BaseModel):
    data: list[float]
    method: str = "mean"
    confidence_threshold: float = 0.95


class VisualizationParameters(BaseModel):
    data: list[float]
    chart_type: str = "line"
    title: str = "Data Visualization"


# Create tool schemas
analysis_schema = ToolSchema(
    name="advanced_analysis", description="Advanced data analysis with confidence threshold", parameters=AnalysisParameters
)

visualization_schema = ToolSchema(
    name="advanced_visualization", description="Advanced data visualization with title", parameters=VisualizationParameters
)


# Use schemas in resource methods
class AdvancedDataAnalysisResource(BaseResource):
    """Example resource with advanced tool calling features."""

    @tool_callable(schema=analysis_schema)
    async def advanced_analysis(self, params: AnalysisParameters) -> dict[str, Any]:
        """Perform advanced data analysis."""
        result = sum(params.data) / len(params.data)
        return {"result": result, "method": params.method, "confidence": params.confidence_threshold}

    @tool_callable(schema=visualization_schema)
    async def advanced_visualization(self, params: VisualizationParameters) -> dict[str, Any]:
        """Create advanced visualization."""
        return {"chart_type": params.chart_type, "title": params.title, "data_points": len(params.data)}

## 4. Error Handling and Validation

Proper error handling and validation are crucial for tool calling. Let's see how to implement this:

In [ ]:
from opendxa import ToolError


class RobustDataAnalysisResource(BaseResource):
    """Example resource with robust error handling."""

    @tool_callable(
        name="robust_analysis",
        description="Robust data analysis with error handling",
        parameters={
            "data": {"type": "array", "description": "List of numerical values"},
            "method": {"type": "string", "description": "Analysis method to use"},
        },
    )
    async def robust_analysis(self, data: list[float], method: str = "mean") -> dict[str, Any]:
        """Perform robust data analysis."""
        try:
            # Validate input
            if not data:
                raise ToolError("No data provided")

            if method not in ["mean", "sum", "median"]:
                raise ToolError(f"Invalid method: {method}")

            # Perform analysis
            if method == "mean":
                result = sum(data) / len(data)
            elif method == "sum":
                result = sum(data)
            else:  # median
                sorted_data = sorted(data)
                n = len(sorted_data)
                result = (sorted_data[n // 2] + sorted_data[-(n // 2 + 1)]) / 2

            return {"result": result, "method": method, "status": "success"}

        except Exception as e:
            raise ToolError(f"Analysis failed: {str(e)}")

## 5. Testing and Debugging

Let's see how to test and debug tool calls:

In [ ]:
import pytest

# Test data
TEST_DATA = [1, 2, 3, 4, 5]


# Test the basic analysis tool
async def test_basic_analysis():
    resource = DataAnalysisResource()
    result = await resource.analyze_data(TEST_DATA, "mean")

    assert "result" in result
    assert "method" in result
    assert result["method"] == "mean"
    assert result["result"] == 3.0


# Test the visualization tool
async def test_visualization():
    resource = DataAnalysisResource()
    result = await resource.visualize_data(TEST_DATA, "line")

    assert "chart_type" in result
    assert "data_points" in result
    assert result["chart_type"] == "line"
    assert result["data_points"] == 5


# Test error handling
async def test_error_handling():
    resource = RobustDataAnalysisResource()

    # Test empty data
    with pytest.raises(ToolError):
        await resource.robust_analysis([], "mean")

    # Test invalid method
    with pytest.raises(ToolError):
        await resource.robust_analysis(TEST_DATA, "invalid")


# Run the tests
if __name__ == "__main__":
    pytest.main([__file__])

## Next Steps

In this tutorial, we've covered:

1. Understanding tool calling in OpenDXA
2. Using tool calling in planning and reasoning
3. Advanced tool calling features
4. Error handling and validation
5. Testing and debugging tool calls

In the next tutorial, we'll explore the MCP (Model Control Protocol) resource in OpenDXA, which provides a standardized way to interact with different types of models.